In [1]:
import langchain

In [4]:
# !pip -q install langchain openai tiktoken chromadb python-dotenv

In [52]:
# !pip install PyMuPDF

In [1]:
import os

from collections import defaultdict
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str='cl100k_base') -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

from dotenv import load_dotenv

load_dotenv()
# os.environ["OPENAI_API_KEY"] = ""

True

In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyMuPDFLoader
from langchain.document_loaders import DirectoryLoader

In [3]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
# loader = DirectoryLoader('./data/', glob="./*.pdf", loader_cls=PyMuPDFLoader)
file_path='data/서울특별시 버스노선 혼잡도 예측을 통한 다람쥐버스 신규 노선제안(장려).pdf'
loader = PyMuPDFLoader(file_path=file_path)
documents = loader.load()

In [43]:
documents[0].page_content

'서울특별시 버스 노선 혼잡도 예측을 통한 다람쥐버스 신규 노선 \n제안\n1. 주제 선정 및 자료분석 배경 (주제 선정의 독창성) \n을지로, 여의도 같은 기업의 밀집도가 높은 지역들의 출퇴근 시간에는 대중교통, 자가용 할 것 없이 모\n든 이동수단의 포화상태를 볼 수 있다. 출퇴근 시간에서의 교통 혼잡도를 줄이기 위해서 대중교통의 이\n용을 적극 홍보하고 있지만, 사실상 버스와 지하철을 타지 못하고 몇 대씩 보내고 난 다음에야 탑승할 \n수 있을 정도로 이용객이 넘쳐난다. 이러한 문제를 해결하기 위해 서울시에서 다람쥐버스의 운행을 시\n작하였다. 다람쥐버스는 서울시에서 운행하는 출퇴근 맞춤버스로 출퇴근 시간대의 특정 구간에서 발생\n하는 차내 혼잡을 해소하기 위해 운행하는 순환형 셔틀버스이다. 다람쥐버스라는 이름은 다람쥐가 쳇바\n퀴 돌듯 짧은 구간을 반복 운행한다고 하여 붙여졌다.\n2017년 6월에 4개의 노선으로 반년 간의 시범 운행을 한 결과 차내 혼잡을 39.5% 완화하는 효과와 동\n시에 해당 구간의 전체 버스 이용객이 16.5% 증가하는 효과를 볼 수 있었다. 이러한 결과를 반영하여 \n2018년에는 3개의 노선이 추가되어 현재는 총 7개의 노선이 운행 중이다.\n다람쥐버스는 현재 이미 시행 중인 정책이지만 2018년에 국토교통부가 주관한 2018 지속가능 교통도시\n평가에서 최우수정책으로 선정될 만큼 긍정적인 평가를 받고 있고 타 지역에서의 운행을 기대하는 시민\n들의 목소리도 높은 것으로 나타났다. 따라서 이번 과제에서 버스 정류소 및 노선별 이용객 수를 예측\n하고 버스 내 혼잡도를 계산함으로써 가장 큰 효과를 얻을 수 있는 신규 노선을 제안하고자 한다. \n2. 분석 내용 (자료분석의 우수성, 데이터 활용성)\n활용데이터\n『서울시 버스노선별, 정류장별, 시간대별 승·하차 인원 정보』 데이터에서 일별, 시간대별 승·하차 인원을 \n구하기 위해 출근 시간대인  7시~9시의 승·하차 인원을 추출, 평균을 취하고 연도별 일수로 나누었다. \n

In [30]:
# https://github.com/openai/tiktoken/blob/main/tiktoken/model.py
# MODEL_TO_ENCODING: dict[str, str] = {
#     # chat
#     "gpt-4": "cl100k_base",
#     "gpt-3.5-turbo": "cl100k_base",
#     "gpt-35-turbo": "cl100k_base",  # Azure deployment name
#     # text
#     "text-davinci-003": "p50k_base",
#     "text-davinci-002": "p50k_base",
#     "text-davinci-001": "r50k_base",
#     "text-curie-001": "r50k_base",
#     "text-babbage-001": "r50k_base",
#     "text-ada-001": "r50k_base",
#     "davinci": "r50k_base",
#     "curie": "r50k_base",
#     "babbage": "r50k_base",
#     "ada": "r50k_base",
#     # code
#     "code-davinci-002": "p50k_base",
#     "code-davinci-001": "p50k_base",
#     "code-cushman-002": "p50k_base",
#     "code-cushman-001": "p50k_base",
#     "davinci-codex": "p50k_base",
#     "cushman-codex": "p50k_base",
#     # edit
#     "text-davinci-edit-001": "p50k_edit",
#     "code-davinci-edit-001": "p50k_edit",
#     # embeddings
#     "text-embedding-ada-002": "cl100k_base",
#     # old embeddings
#     "text-similarity-davinci-001": "r50k_base",
#     "text-similarity-curie-001": "r50k_base",
#     "text-similarity-babbage-001": "r50k_base",
#     "text-similarity-ada-001": "r50k_base",
#     "text-search-davinci-doc-001": "r50k_base",
#     "text-search-curie-doc-001": "r50k_base",
#     "text-search-babbage-doc-001": "r50k_base",
#     "text-search-ada-doc-001": "r50k_base",
#     "code-search-babbage-code-001": "r50k_base",
#     "code-search-ada-code-001": "r50k_base",
#     # open source
#     "gpt2": "gpt2",
# }

In [41]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='p50k_base',
    chunk_size=1000, chunk_overlap=100,
    add_start_index=True)
docs = text_splitter.split_documents(documents)

In [42]:
docs

[Document(page_content='서울특별시 버스 노선 혼잡도 예측을 통한 다람쥐버스 신규 노선 \n제안\n1. 주제 선정 및 자료분석 배경 (주제 선정의 독창성) \n을지로, 여의도 같은 기업의 밀집도가 높은 지역들의 출퇴근 시간에는 대중교통, 자가용 할 것 없이 모\n든 이동수단의 포화상태를 볼 수 있다. 출퇴근 시간에서의 교통 혼잡도를 줄이기 위해서 대중교통의 이\n용을 적극 홍보하고 있지만, 사실상 버스와 지하철을 타지 못하고 몇 대씩 보내고 난 다음에야 탑승할 \n수 있을 정도로 이용객이 넘쳐난다. 이러한 문제를 해결하기 위해 서울시에서 다람쥐버스의 운행을 시\n작하였다. 다람쥐버스는 서울시에서 운행하는 출퇴근 맞춤버스로 출퇴근 시간대의 특정 구간에서 발생\n하는 차내 혼잡을 해소하기 위해 운행하는 순환형 셔틀버스이다. 다람쥐버스라는 이름은 다람쥐가 쳇바\n퀴 돌듯 짧은 구간을 반복 운행한다고 하여 붙여졌다.', metadata={'source': 'data/서울특별시 버스노선 혼잡도 예측을 통한 다람쥐버스 신규 노선제안(장려).pdf', 'file_path': 'data/서울특별시 버스노선 혼잡도 예측을 통한 다람쥐버스 신규 노선제안(장려).pdf', 'page': 0, 'total_pages': 5, 'format': 'PDF 1.4', 'title': '', 'author': 'user', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.9139', 'producer': 'Hancom PDF 1.3.0.538', 'creationDate': "D:20200724181854+09'00'", 'modDate': "D:20200724181854+09'00'", 'trapped': '', 'start_index': 0}),
 Document(page_content='퀴 돌듯 짧은 구간을 반복 운행한다고 하여 붙여졌다.\n2017년 6월에 4개의 노선으로 반년 간의 시범 운행을 한 결과 차

In [5]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

In [6]:
docs

[Document(page_content='서울특별시 버스 노선 혼잡도 예측을 통한 다람쥐버스 신규 노선 \n제안\n1. 주제 선정 및 자료분석 배경 (주제 선정의 독창성) \n을지로, 여의도 같은 기업의 밀집도가 높은 지역들의 출퇴근 시간에는 대중교통, 자가용 할 것 없이 모\n든 이동수단의 포화상태를 볼 수 있다. 출퇴근 시간에서의 교통 혼잡도를 줄이기 위해서 대중교통의 이\n용을 적극 홍보하고 있지만, 사실상 버스와 지하철을 타지 못하고 몇 대씩 보내고 난 다음에야 탑승할 \n수 있을 정도로 이용객이 넘쳐난다. 이러한 문제를 해결하기 위해 서울시에서 다람쥐버스의 운행을 시\n작하였다. 다람쥐버스는 서울시에서 운행하는 출퇴근 맞춤버스로 출퇴근 시간대의 특정 구간에서 발생\n하는 차내 혼잡을 해소하기 위해 운행하는 순환형 셔틀버스이다. 다람쥐버스라는 이름은 다람쥐가 쳇바\n퀴 돌듯 짧은 구간을 반복 운행한다고 하여 붙여졌다.\n2017년 6월에 4개의 노선으로 반년 간의 시범 운행을 한 결과 차내 혼잡을 39.5% 완화하는 효과와 동\n시에 해당 구간의 전체 버스 이용객이 16.5% 증가하는 효과를 볼 수 있었다. 이러한 결과를 반영하여 \n2018년에는 3개의 노선이 추가되어 현재는 총 7개의 노선이 운행 중이다.\n다람쥐버스는 현재 이미 시행 중인 정책이지만 2018년에 국토교통부가 주관한 2018 지속가능 교통도시\n평가에서 최우수정책으로 선정될 만큼 긍정적인 평가를 받고 있고 타 지역에서의 운행을 기대하는 시민\n들의 목소리도 높은 것으로 나타났다. 따라서 이번 과제에서 버스 정류소 및 노선별 이용객 수를 예측\n하고 버스 내 혼잡도를 계산함으로써 가장 큰 효과를 얻을 수 있는 신규 노선을 제안하고자 한다. \n2. 분석 내용 (자료분석의 우수성, 데이터 활용성)\n활용데이터\n『서울시 버스노선별, 정류장별, 시간대별 승·하차 인원 정보』 데이터에서 일별, 시간대별 승·하차 인원을 \n구하기 위해 출근 시간대인  7시~9시의 승·하차 인원을 추출, 평

In [11]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

In [12]:
docs

[Document(page_content='서울특별시 버스 노선 혼잡도 예측을 통한 다람쥐버스 신규 노선 \n제안\n1. 주제 선정 및 자료분석 배경 (주제 선정의 독창성) \n을지로, 여의도 같은 기업의 밀집도가 높은 지역들의 출퇴근 시간에는 대중교통, 자가용 할 것 없이 모\n든 이동수단의 포화상태를 볼 수 있다. 출퇴근 시간에서의 교통 혼잡도를 줄이기 위해서 대중교통의 이\n용을 적극 홍보하고 있지만, 사실상 버스와 지하철을 타지 못하고 몇 대씩 보내고 난 다음에야 탑승할 \n수 있을 정도로 이용객이 넘쳐난다. 이러한 문제를 해결하기 위해 서울시에서 다람쥐버스의 운행을 시\n작하였다. 다람쥐버스는 서울시에서 운행하는 출퇴근 맞춤버스로 출퇴근 시간대의 특정 구간에서 발생\n하는 차내 혼잡을 해소하기 위해 운행하는 순환형 셔틀버스이다. 다람쥐버스라는 이름은 다람쥐가 쳇바\n퀴 돌듯 짧은 구간을 반복 운행한다고 하여 붙여졌다.\n2017년 6월에 4개의 노선으로 반년 간의 시범 운행을 한 결과 차내 혼잡을 39.5% 완화하는 효과와 동\n시에 해당 구간의 전체 버스 이용객이 16.5% 증가하는 효과를 볼 수 있었다. 이러한 결과를 반영하여 \n2018년에는 3개의 노선이 추가되어 현재는 총 7개의 노선이 운행 중이다.\n다람쥐버스는 현재 이미 시행 중인 정책이지만 2018년에 국토교통부가 주관한 2018 지속가능 교통도시\n평가에서 최우수정책으로 선정될 만큼 긍정적인 평가를 받고 있고 타 지역에서의 운행을 기대하는 시민\n들의 목소리도 높은 것으로 나타났다. 따라서 이번 과제에서 버스 정류소 및 노선별 이용객 수를 예측\n하고 버스 내 혼잡도를 계산함으로써 가장 큰 효과를 얻을 수 있는 신규 노선을 제안하고자 한다. \n2. 분석 내용 (자료분석의 우수성, 데이터 활용성)\n활용데이터\n『서울시 버스노선별, 정류장별, 시간대별 승·하차 인원 정보』 데이터에서 일별, 시간대별 승·하차 인원을 \n구하기 위해 출근 시간대인  7시~9시의 승·하차 인원을 추출, 평

In [85]:
lenTokenByPage = defaultdict()
for doc in docs:
    lenTokenByPage[doc.metadata['page']] = num_tokens_from_string(doc.page_content)

In [86]:
lenTokenByPage

defaultdict(None, {0: 512, 1: 570, 2: 899, 3: 187, 4: 951, 5: 166})

In [38]:
texts = [doc.dict() for doc in docs]

In [56]:
texts[0]

{'page_content': '지역의 사회구조적 특성이 빈집 형성에 미친 영향에 관한 \n패널분석\n1. 주제 선정 및 자료분석 배경 (주제 선정의 독창성) \n ◦ [연구의 배경] 빈집 증가 추세의 심화  \n   - 최근 빈집 증가로 「빈집 및 소규모주택 정비에 관한 특례법」이 제정되는 등 빈집\n에 대한 사회적․정책적 관심이 높아진 상황임\n   - 빈집이 증가하면서 안전사고나 범죄, 도시미관이나 주거환경에의 장해 발생을 우려\n하는 시각도 존재함(남지현, 2015) \n ◦ [연구의 필요성] 빈집 관련 실증적 연구의 필요성\n   - 빈집 정비와 활용을 목적으로 하는 사업들이 실효적으로 추진되기 위해서는 빈집의\n실태를 파악하는 한편, 빈집 발생에 영향을 미치는 요인들에 대한 실증적 연구가\n긴요함\n   - 농촌지역뿐만 아니라 도시지역 안에서도 빈집이 증가하고 있는 상황은 단순히 저출\n산․고령화의 여파가 아니라 지역이 지닌 사회구조적 취약성이 장기간 누적된 결과\n로서 이해되어야 할 필요성을 시사함\n ◦ [선행연구 검토] 자료의 범위 및 분석단위 설정에서의 한계\n   - 빈집을 주제로 한 선행연구들은 단년도 자료에 바탕하고 있어 시간적 흐름에 따라\n빈집 발생에 미치는 요인의 영향력을 면밀하게 분석하기 어려움(예컨대, 김현중․한\n홍구․여관현, 2018; 노민지․유선종, 2016)\n   - 빈집 관련 선행연구에서 고려된 변수는 인구․가구․주택과 관련된 것으로 제한되어\n있어, 폐업 사업체수와 같은 지역의 경제적 상황이나 병원수 등과 같은 생활편의\n(amenity) 수준 등 빈집 형성에 영향을 미치는 생태학적 요인을 다양하게 고려하는\n것이 필요함\n   - 또한 빈집 관련 선행연구들은 자료의 분석단위를 기초자치단체(시․군․구)로 다소 넓\n게 설정하거나 한 개 동과 같이 보다 작은 범위(예컨대, 전영미․김세훈, 2016; 오승\n훈, 2015)에 대해 사례검토 하는 것으로 양분화되고 있어 전국적․보편적 논의로 확\n대하기에는 일정한 한계가 

In [36]:
_len = sum([len(doc.page_content) for doc in docs])

In [37]:
_len

17385

In [109]:
## Embedding

In [87]:
## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings()

In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

## Here is the nmew embeddings being use
embedding = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                          model_kwargs={"device": "cuda"})

## Create a Vector DB

In [33]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

vectordb = Chroma.from_documents(documents=docs, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

AttributeError: 'str' object has no attribute 'page_content'

In [55]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [56]:
# Now we can load the persisted database from disk, and use it as normal. 
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

## Make a retriever

In [86]:
# retriever = vectordb.as_retriever()
retriever = vectordb.as_retriever(search_kwargs={"k": 2})
# retriever.search_type
# retriever.search_kwargs

In [87]:
docs = retriever.get_relevant_documents("과거는 반복된다")

In [88]:
len(docs)

2

In [89]:
docs

[Document(page_content='인 차원으로 풀이되어 옴에 따라 읍면동과 같은 소지역 단위에서의 빈집정비 방향\n설정에 유효한 설명이 부족하였음. 이상의 분석결과는 같은 지자체 안에서도 빈집\n발생 정도에 차이가 나타나는 이유를 설명하는 틀을 제공하며, 빈집 발생을 완화하\n기 위해 취할 수 있는 정책적 방안 모색을 가능케 함\n  ◦ 빈집 발생 및 증가 현상에 대한 이해 증대\n   - 분석결과 읍면동 내 빈집 발생 및 증가 현상은 인구구조 특성뿐만 아니라 지역의\n경제적 상황, 생활편의 등 지역생태학적 요인이 복합적으로 작용한 결과임. 단순히\n중앙 및 지방자치단체기관이 지방으로 이전되었다고 해서 빈집 발생이 억제되는 효\n과로 이어지지 않으며, 분석결과에서도 자치단체기관수가 빈집 발생에 유의한 영향\n을 미치지 않는 것으로 나타남. 또한 도시 지역의 빈집 발생은 폐업 사업체수 등\n지역의 경제적 상황과 밀접히 관련되어 있다는 분석결과는 최근 대기업 두 곳이 연\n이어 철수한 군산 등 전북 지역의 빈집 발생 심화현상에 적확한 설명을 제공함', metadata={'source': 'data/지역의 사회구조적 특성이 빈집 형성에 미친 영향에 관한 패널 분석(우수).pdf', 'file_path': 'data/지역의 사회구조적 특성이 빈집 형성에 미친 영향에 관한 패널 분석(우수).pdf', 'page': 4, 'total_pages': 6, 'format': 'PDF 1.4', 'title': '', 'author': 'mhan', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.9139', 'producer': 'Hancom PDF 1.3.0.538', 'creationDate': "D:20200724181319+09'00'", 'modDate': "D:20200724181319+09'00'", 'trapped': ''}),
 Document(page_content='일수록 유의하게 빈집이 

## Make a Chin

In [105]:
# create the chain to answer questions 
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [101]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


In [ ]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [106]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    # print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [107]:
# full example
query = "사회구조적 특성이 빈집 형성에 끼친 영항에 대해 설명해주세요"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 
사회구조적 특성이 빈집 형성에 끼친 영향은 다양합니다. 예를 들어, 사회구조가 공동체 중심적이라면, 가구가 생기거나 사람들이 이사하는 경우 빈집이 생기기가 쉽습니다. 반면, 사회구조가 개인 중심적이라면, 빈집이


Sources:


## Deleteing the DB

In [97]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# delete the directory
!rm -rf db/

## Starting again loading the db

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [99]:
persist_directory = 'db'
embedding = OpenAIEmbeddings()

vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding,
                   )

retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [ ]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [ ]:
# create the chain to answer questions 
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm, 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

In [ ]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])